# Steganografija slike

Način na koji se može sakriti tajna poruka tokom slanja tako da ne privlači pažnju napadača jeste steganografija. U kriptografiji poruke su šifrovane te često zainteresuju napadače koji pokušavaju da razbiju šifru, dok u steganografiji sakrivena poruka se ne zaključava vidljivim ključem koji se potom odvojeno šalje, pa ne postoji objekat privlačenja pogleda napadača.
<hr>
<i>Steganografija</i> je metoda sakrivanja tajne poruke u slici, zvuku, videu itd koji će biti prenosni mediji. Kako tekstualna poruka, tako se i slika može sakriti u drugoj slici i slično. Takvom steganografijom ćemo se baviti u nastavku.

Tehnika sakrivanja poruke/slike u slici-nosaču jeste ubacivanje značajnih bitova (bitova velike težine) tajne poruke/slike na mestima najnižih bitova slike-nosača, jer na taj način slika-nosač neće biti vidljivo promenjena i tajna poruka/slika neće biti vidljiva. Bitno je izabrati dobru sliku-nosač koja je puna detalja, jer male promene u pikselima koji predstavljaju oštre ivice mogu biti vidljive. Dakle, ono što će biti promenjeno jeste intenzitet RGB boja u nekim pikselima. Procenat originalne slike koji možemo menjati je mali, 10-ak procenata da promena slike ne bi bila vidljiva. 

<hr>
Prikazivanje svakog piksela je moguće čuvanjem vrednosti boja u njemu. Koristićemo RGB model boja. Svaki piksel će imati tri 8-bitne vrednosti za svaku od boja.

<img src="img\rgb.png">
<hr>
<img src="img\tajno.png" width="500">

Kako bismo dobili male promene slike-nosača menjaćemo vrednosti njenih najnižih bitova sa bitovima najveće težine poruke ili slike koju želimo da sakrijemo. 

Svaki piksel će biti predstavljen trojkama 8-bitnih brojeva, svaki će predstavljati jačinu jedne od RGB boja. Vrednosti idu od 0 do 255.
<img src="img\rgb_bit.png" width="500">

Za početak kreirajmo funkcije za sakrivanje slike unutar druge slike. Naravno, odvojene su i neke propratne funkcije radi lakseg razumevanja koda.

In [1]:
%config IPCompleter.greedy=True

In [121]:
from PIL import Image
import numpy as np

def copy_image(image):
    return Image.new(image.mode, image.size, (0,0,0))

def get_size(image):
    return (image.size[0], image.size[1])


class Steganography_image:
    #From 3 given numbers this returns rgb values in binary representation
    get_binary_rgb = lambda r, g, b: (format(r, "b").zfill(8), format(g, "b").zfill(8), (format(b, "b").zfill(8)))
  
    @staticmethod
    def get_rgb(image_rgb):
         return Steganography_image.get_binary_rgb(image_rgb[0], image_rgb[1], image_rgb[2])

    
    #Method that returns new image that is a result of merging secret image 
    #into holder image that will be send and visible
    @staticmethod
    def encrypt(image_holder, image_secret):
        #If image size (with and hight given as a tuple) is bigger then holder image, than we want to exit
        h_weight, h_hight = get_size(image_holder)
        s_weight, s_hight = get_size(image_secret)
    
        if s_weight > h_weight or s_hight > h_hight:
            raise ValueError('Secret image is bigger than holder image!')
        
        #We can see RGB values for all pixcels with command: #print np.asarray(image_holder)
        
        #Pixel maps for both images
        image_holder_pixels = image_holder.load()
        image_secret_pixels = image_secret.load()
       
        #Output image will be the same as holder, and then we are going to change it's pixels
        #importing part of pixels of secret image
        output_image = copy_image(image_holder)
        output_image_pixels = output_image.load()
        
        for i in range(h_weight):
            for j in range(h_hight): 
                rgb_binary_holder = Steganography_image.get_rgb(image_holder_pixels[i,j])
                 
                #Check if the secret image is finished because it is smaller than holder image         
                if(s_weight > i and s_hight > j):
                    rgb_binary_secret = Steganography_image.get_rgb(image_secret_pixels[i,j])
                else:
                    rgb_binary_secret = Steganography_image.get_rgb((0, 0, 0))
                
                #Now we have rgb binary values for holder and secret image
                #Then we want to place secret image into holder one into last 4 bits
                r = rgb_binary_holder[0][:4] + rgb_binary_secret[0][:4]
                g = rgb_binary_holder[1][:4] + rgb_binary_secret[1][:4]
                b = rgb_binary_holder[2][:4] + rgb_binary_secret[2][:4]
                output_image_pixels[i, j] = (int(r, 2), int(g, 2), int(b, 2))
        
        image_holder.close()
        image_secret.close()        
        return output_image
        
    
    #Method that returns secret image that is hidden in visible recived image
    @staticmethod
    def decrypt(image):
        #Pixel map for image
        image_pixels = image.load()
      
        weight, hight = get_size(image)
        size = image.size
        
        #Output image will be the same as holder, and then we are going to change it's pixels
        #importing part of pixels of secret image only. After that we will crop extra pixels
        output_image = copy_image(image)  #Image.new(image.mode, size, (0, 0, 0))
        output_image_pixels = output_image.load()
        
        #Variables for the size of secret image
        k = 0
        l = 0

       #Making rgb map and checking if all pixels are zero then there is no hidden picture
        for i in range(weight):
            for j in range(hight): 
                rgb_binary = Steganography_image.get_rgb(image_pixels[i,j])
                
                #Now we have rgb binary values
                #Then we want to get secret image from holder image by taking last 4 bits
                r = rgb_binary[0][4:] + '0000'
                g = rgb_binary[1][4:] + '0000'
                b = rgb_binary[2][4:] + '0000'
                output_image_pixels[i, j] = (int(r, 2), int(g, 2), int(b, 2))
                
                #If all parts are zero, then there is no secret picture merged on that pixel.
                #We are seting size of secret picture, last pixel that is not zero is end of the secret image
                #so we are saving that pixel indexes in k and l.
                if output_image_pixels[i, j] == (0, 0, 0):
                    pass
                else:
                    k = i
                    l = j
                    
        output_weight = k + 1
        output_hight = l + 1
        #Crop picture from left upper corner to size that we found (size of secret image)
        output_image = output_image.crop((0, 0, output_weight, output_hight))               
        
        image.close()
        return output_image

Sada možemo da isprobamo funkcije. Najpre uzećemo kao sliku koju ćemo slati (koja će biti vidljiva svima) <b>New_york</b> koja ima dosta detalja na sebi te promene koje algoritam zahteva neće biti vidljive! Sliku koju želimo da sakrijemo od posmatrača i da je krišom pošaljemo je <b>secret_photo</b>. Ona će biti sakrivena unutar prve slike. 

Pokrenimo prvi deo pripremanja slike za slanje na sledeći način:

In [122]:
encrypt = Steganography_image.encrypt(Image.open('New_york.png'), Image.open('secret_photo.png'))       
encrypt.save('res\output.png')

Tako smo kreirali sliku <b>output</b> koja predstavlja sliku u slici, spremnu za sigurno slanje primaocu.

Na prvi pogled slike su identične i izgledaće ovako:

<tr>
<td> <img src="img\New_york_before.png"> </td>
<td> <img src="img\New_york_after.png"> </td>
</tr>
<tr>
<td style="text-align:center"> Originalna slika </td>
<td style="text-align:center"> Slika u koju je sakrivena slika koju želimo da pošaljemo </td>
</tr>

Sliku koju dobijemo možemo bez problema da pošaljemo, napadač neće moći da vidi da je ta slika samo posrednik!

Kada slika stigne primaocu, on je može drugom funkcijom raspakovati tako da zapravo dobije sliku koju mu je pošiljaoc namenio, i to pokretanjem sledećeg koda:

In [53]:
decrypt = Steganography_image.decrypt(Image.open('res\output.png'))       
decrypt.save('res\output_secret.png')

Dobijamo sliku <b>output</b> koja predstavlja sakrivenu sliku koju je pošiljaoc želeo tajno da dostavi primaocu.

Opet je na prvi pogled slika identična, a kada se malo bolje zagledamo vidimo sliku lošijeg kvaliteta. Primaoc će i sa tim minusom u ovom algoritmu videti dosta dobro poruku pošiljaoca.

Razliku vidimo ispod:

<tr>
<td> <img src="img\secret_photo_before.png"> </td>
<td> <img src="img\secret_photo_after.png"> </td>
</tr>
<tr>
<td style="text-align:center"> Originalna slika koju pošiljaoc želi da pošalje </td>
<td style="text-align:center"> Slika nakon što je primaoc raspakuje </td>
</tr>

<hr>
Sledeće što želimo da omogućimo jeste slanje tajnog teskta. To ćemo uraditi na sličan način, sakrivajući tekst unutar slike koju šaljemo.

In [13]:
#ONLY FOR ASCII CHARATERS
class Steganography_text:
    #Returns binary representation from text
    get_binary_text = lambda num : (format(ord(num), '08b'))
    
    #Returns ascii from binary representation
    def get_ascii(binary):
        number = int(binary, 2)
        return number.to_bytes((number.bit_length() + 7) // 8, 'big').decode()
    
    #Method that returns image with not visible hidden message
    @staticmethod
    def encrypt(image_name, message):
        image = Image.open(image_name + '.png')
        image_pixels = image.load()
        
        #Generating binary representation of whole text
        message_bin = []
        message_len = len(message)
        for i in range(message_len):
            message_bin.append(Steganography_text.get_binary_text(message[i]))
        
        #Output image with secret text is the same as forwarded image at first
        output_image = copy_image(image)
        output_image_pixels = output_image.load()
        weight, hight = get_size(output_image)
        
        #k is counter for number of letters in message
        k = 0
        for i in range(weight):
            for j in range(hight):
                rgb_binary = Steganography_image.get_rgb(image_pixels[i,j])
                
                #If message is not finished take binary representation of letter, 
                #else zero binary pixel.
                #Representation of the text made like this is getting into image
                if(message_len > 0):
                    rgb_binary_secret =  message_bin[k]
                else:
                    rgb_binary_secret = '00000000'
                
                #In one pixel of image we are putting one letter in parts (3, 3 and 2 bits) 
                #at the end of each rgb part of the image, creating new rgb presentation
                r = rgb_binary[0][:5] + rgb_binary_secret[:3]
                g = rgb_binary[1][:5] + rgb_binary_secret[3:6]
                b = rgb_binary[2][:6] + rgb_binary_secret[6:8]
                output_image_pixels[i, j] = (int(r, 2), int(g, 2), int(b, 2))
                    
                k += 1
                message_len -= 1
                    
                output_image_pixels[i, j] = (int(r, 2), int(g, 2), int(b, 2)) 
                    
        return output_image
    
    
    #Method that returns secret message hidden into image
    @staticmethod
    def decrypt(image):
        #Pixel map for image
        image = Image.open(image + '.png')
        image_pixels = image.load()
      
        weight, hight = get_size(image)
        size = image.size
        
        #Output is the message
        message = ""
          
        #If, after manipulating, pixel is zero then there is no secret message into that pixel
        for i in range(weight):
            for j in range(hight): 
                rgb_binary = Steganography_image.get_rgb(image_pixels[i,j])
                
                #Now we want to take part of the pixels that will maybe be letter from message
                m = rgb_binary[0][5:] + rgb_binary[1][5:] + rgb_binary[2][6:]
                
                #If new pixel is zero, then there is no secret message
                if m != '00000000':
                     message += Steganography_text.get_ascii(m)
                    
        return message

Isprobajmo kod tako što ćemo unutar slike <b>City</b> da sakrijemo poruku koja glasi <b>"Veoma vazno! Vandredno stanje je gotovo!"</b> kako bismo novu sliku sa porukom poslali nekome tako da poruka ne bude viljiva.


Pokretanjem sledećeg koda dobijamo sliku kao rezultat, spremnu za slanje:

In [54]:
message = "Veoma vazno! Vandredno stanje je gotovo!"
encrypt = Steganography_text.encrypt('City', message)
encrypt.save("res\City_message.png")

Uporedjivanjem originalne i dobijene slike ne možemo uočiti nikakvu razliku:

<tr>
<td> <img src="img\City_before.png"> </td>
<td> <img src="img\City_after.png"> </td>
</tr>
<tr>
<td style="text-align:center"> Originalna slika koju pošiljaoc želi da pošalje </td>
<td style="text-align:center"> Slika koja sadrži tajnu poruku, spremna za slanje </td>
</tr>

Nakon što sliku dobije primaoc, raspakivanjem može da pročita skriveni tekst jednostavnom komandom:

In [55]:
Steganography_text.decrypt('res\City_message')

'Veoma vazno! Vandredno stanje je gotovo!'

Kao rezultat dobili smo poruku koja je upravo i očekivana: <b>"Veoma vazno! Vandredno stanje je gotovo!"</b> 
</hr>
Na ovaj način možemo da šaljemo poruke i slike unutar neke druge slike koja će to prkriti. Način na koji se kod može razviti je promenljive prirode, može varirati u zavisnosti od ideje i time napadaču otežati posao u kreiranju koda koji razbija poslatu sliku kako bi dobio tajnu poruku. Naravno, napadaču najpre mora pasti na pamet da je poslata slika zapravo osnova za sakrivanje neke tajne poruke.

<hr>
Ubacimo sada još neke primere, najpreako su slika nosač i tajna slika iste veličine. Koristićemo sledeće slike:

<tr>
<td> <img src="img\New_york_before2.png"> </td>
<td> <img src="img\secret_photo_before2.png"> </td>
</tr>
<tr>
<td style="text-align:center"> Slika nosač kao i u prošlom primeru, s tim da su joj izmenjene dimenzije (iste će biti veličine kao tajna slika) </td>
<td style="text-align:center"> Tajna slika koju želimo da sakrijemo </td>
</tr>

In [123]:
secret_image = Image.open('secret_photo2.png')
#Changing size of image
holder_image = Image.open('New_york.png').resize(secret_image.size)

encrypt = Steganography_image.encrypt(holder_image, secret_image)
encrypt.save('res\output_same_size.png')

Uporedimo razlike slike nosača pre i posle ubacivanja tajne slike.

<tr>
<td> <img src="img\New_york_before2.png"> </td>
<td> <img src="img\New_york_after2.png"> </td>
</tr>
<tr>
<td style="text-align:center"> Originalna slika </td>
<td style="text-align:center"> Slika u koju je sakrivena tajna slika </td>
</tr>

Nakon dekripcije možemo uporediti i tajnu sliku.

In [67]:
decrypt = Steganography_image.decrypt(Image.open('res\output_same_size.png'))       
decrypt.save('res\output_secret_same_size.png')

<tr>
<td> <img src="img\secret_photo_before2.png"> </td>
<td> <img src="img\secret_photo_after2.png"> </td>
</tr>
<tr>
<td style="text-align:center"> Tajna slika pre enkripcije </td>
<td style="text-align:center"> Tajna slika posle dekripcije </td>
</tr>

Ideja je ista, dobija se slika dovoljno dobra, sa malo oštrijim ivicama tj. pikseli su uočljiviji.

<hr>
<hr>
Sledeći primer će predstavljati sakrivanje tajne slike u sliku nosač koja će biti malog kolariteta. Svakako, kao što je na početku spomenuto, važno je da slika nosač ima dosta detalja kako ne bi bila vidljiva bilo koja izmena piksela. 

<tr>
<td> <img src="img\SimpleBackground_before.png"> </td>
</tr>
<tr>
<td style="text-align:center"> Slika nosač </td>
</tr>

In [124]:
encrypt = Steganography_image.encrypt(Image.open('SimpleBackground.png'), Image.open('secret_photo.png'))       
encrypt.save('res\output_simpe.png')

Možemo sada videti sliku nosač posle enkripcije. Ako se bolje zagledamo vidimo dosta izmenjenih piksela i deo nase tajne slike golim okom. Biranjem ovako loše slike nosača (bez mnogo detalja) i slanjem takve slike kao poruku izazivamo napadače da razmisle o tako poslatoj slici. Lako bi ih zainteresovala ovakva slika i zapitali bi se šta se tu krije.

<tr>
<td> <img src="img\SimpleBackground_after.png"> </td>
</tr>

Kada bismo dekriptovali sada ovakvu sliku dobili bismo dobru tajnu sliku, na isti način kao i do sada.

In [125]:
decrypt = Steganography_image.decrypt(Image.open('res\output_simpe.png'))       
decrypt.save('res\output_secret_simple.png')

<tr>
<td> <img src="img\secret_photo_before.png"> </td>
<td> <img src="img\secret_photo_after3.png"> </td>
</tr>
<tr>
<td style="text-align:center"> Tajna slika pre enkripcije na lošoj slici nosaču </td>
<td style="text-align:center"> Tajna slika posle dekripcije </td>
</tr>

<hr>
<hr>
Još jedan zanimljiv primer je kada je u pitanju tajna slika veoma velikog kolariteta. Za sliku nosač biramo dobru sliku, sa dosta detalja (jer smo u prethodnom primeru videli da izbor jednostavne slike nosača nikako nam ne odgovara). Uzećemo sliku Njujorka kao sliku nosač, dok nam je tajna slika sledeća:

<tr>
<td> <img width="500" src="img\secret_photo_before5.png"> </td>
</tr>
<tr>
<td style="text-align:center"> Tajna slika velikog kolariteta </td>
</tr>

In [128]:
encrypt = Steganography_image.encrypt(Image.open('New_york.png'), Image.open('secret_photo5.png'))       
encrypt.save('res\output_colorful.png')

Pogledajmo sada sliku nosač nakon ubacivanja ove veoma šarene mandale:

<tr>
<td> <img src="img\output_colorful_after.png"> </td>
</tr>
<tr>
<td style="text-align:center"> Slika nosač posle ubacivanja mandale </td>
</tr>

Vidimo da je slika Njujorka i dalje veoma dobro očuvana, da prilikom ubacivanja slike šarene mandale slika nosač nema vidljive izmene ili sumnjive delove slike, primamljive za napadača.

<hr>
Šta zaključujemo nakon ovih primera?

Zaključujemo da je najvažnije imati u vidu da se slika nosač mora pažljivo birati, tako da ima dosta detalja i boja i da bude svakako veća od slike koju želimo da sakrijemo. To je zapravo jedino na šta korisnik ovog algoritma treba obraiti pažnju kako bi što sigurnije poslao skrivenu sliku ili tekst.

<b>Literatura:</b>
